In [ ]:
import assemblyai as aai

aai.settings.api_key = "fc2ff7e25bc8447ea67c3f1d86ea8f5d"

# audio_file = "./local_file.mp3"
audio_file = r"Data\response (1).wav"
    
config = aai.TranscriptionConfig(speech_model=aai.SpeechModel.best)

transcript = aai.Transcriber(config=config).transcribe(audio_file)

if transcript.status == "error":
  raise RuntimeError(f"Transcription failed: {transcript.error}")

print(transcript.text)

Adab digital Nayat Koch apke Classroom culture ki bhatkare to wahabi kuch bhatiri ki gunjai bachon ki dharmiyan tavun or ichtemai amalko barhava deniki koshishkare mislan group work Kezari jaha bache milkar javabat KE javab de or Apni raika tabad lakare tafani mahal bhi banai mislan kahaniya sunane kevakt ab talaba kobhi shamil kare bahatari kijanip gamzan rahe shukriya subhanallahi hamdi shukriya spanallah.


## AZURE

In [ ]:
import os
import time
import azure.cognitiveservices.speech as speechsdk

def speech_to_text_with_diarization_from_file(audio_file_path, language="ur-IN"):
    """
    Transcribe audio from a file with speaker diarization
    
    Parameters:
        audio_file_path (str): Path to the audio file (.wav format recommended)
        language (str): The language code to use (default: ur-IN for Urdu)
    """
    # Get speech key and region from environment variables
    speech_key = os.environ.get('SPEECH_KEY')
    speech_region = os.environ.get('SPEECH_REGION')
    
    if not speech_key or not speech_region:
        print("Error: Missing SPEECH_KEY or SPEECH_REGION environment variables")
        print("Please set them with your Azure Speech Service credentials")
        return
        
    # Configure speech recognition
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = language
    
    # Create audio configuration from file
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file_path)
    
    # Create conversation transcriber
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
        speech_config=speech_config, 
        audio_config=audio_config
    )
    
    # Enable diarization for intermediate results
    conversation_transcriber.properties.set_property(
        speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults,
        "true"
    )
    
    # Setup result handlers
    done = False
    
    def handle_final_result(evt):
        speaker = evt.result.user_id if evt.result.user_id else "Unknown"
        print(f"FINAL: Speaker {speaker}: {evt.result.text}")
    
    def handle_intermediate_result(evt):
        speaker = evt.result.user_id if evt.result.user_id else "Unknown"
        print(f"INTERIM: Speaker {speaker}: {evt.result.text}")
    
    def handle_session_stopped(evt):
        nonlocal done
        print(f"Session stopped: {evt}")
        done = True
    
    def handle_canceled(evt):
        nonlocal done
        print(f"CANCELED: {evt.cancellation_details.reason}")
        if evt.cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: Error Code={evt.cancellation_details.error_code}")
            print(f"CANCELED: Error Details={evt.cancellation_details.error_details}")
        done = True
    
    # Connect callbacks
    conversation_transcriber.transcribed.connect(handle_final_result)
    conversation_transcriber.transcribing.connect(handle_intermediate_result)
    conversation_transcriber.session_stopped.connect(handle_session_stopped)
    conversation_transcriber.canceled.connect(handle_canceled)
    
    # Start continuous recognition
    print(f"Starting transcription with speaker diarization for language: {language}")
    conversation_transcriber.start_transcribing_async()
    
    # Wait for completion or user interrupt
    try:
        while not done:
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("Stopping transcription...")
    finally:
        conversation_transcriber.stop_transcribing_async()


def speech_to_text_with_diarization_from_mic(language="ur-IN"):
    """
    Transcribe audio from microphone with speaker diarization
    
    Parameters:
        language (str): The language code to use (default: ur-IN for Urdu)
    """
    # Get speech key and region from environment variables
    speech_key = os.environ.get('SPEECH_KEY')
    speech_region = os.environ.get('SPEECH_REGION')
    
    if not speech_key or not speech_region:
        print("Error: Missing SPEECH_KEY or SPEECH_REGION environment variables")
        print("Please set them with your Azure Speech Service credentials")
        return
        
    # Configure speech recognition
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = language
    
    # Create audio configuration for microphone
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    
    # Create conversation transcriber
    conversation_transcriber = speechsdk.transcription.ConversationTranscriber(
        speech_config=speech_config, 
        audio_config=audio_config
    )
    
    # Enable diarization for intermediate results
    conversation_transcriber.properties.set_property(
        speechsdk.PropertyId.SpeechServiceResponse_DiarizeIntermediateResults,
        "true"
    )
    
    # Setup result handlers
    done = False
    
    def handle_final_result(evt):
        speaker = evt.result.user_id if evt.result.user_id else "Unknown"
        print(f"FINAL: Speaker {speaker}: {evt.result.text}")
    
    def handle_intermediate_result(evt):
        speaker = evt.result.user_id if evt.result.user_id else "Unknown"
        print(f"INTERIM: Speaker {speaker}: {evt.result.text}")
    
    def handle_session_stopped(evt):
        nonlocal done
        print(f"Session stopped: {evt}")
        done = True
    
    def handle_canceled(evt):
        nonlocal done
        print(f"CANCELED: {evt.cancellation_details.reason}")
        if evt.cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"CANCELED: Error Code={evt.cancellation_details.error_code}")
            print(f"CANCELED: Error Details={evt.cancellation_details.error_details}")
        done = True
    
    # Connect callbacks
    conversation_transcriber.transcribed.connect(handle_final_result)
    conversation_transcriber.transcribing.connect(handle_intermediate_result)
    conversation_transcriber.session_stopped.connect(handle_session_stopped)
    conversation_transcriber.canceled.connect(handle_canceled)
    
    # Start continuous recognition
    print(f"Starting microphone transcription with speaker diarization for language: {language}")
    print("Speak into your microphone...")
    print("Press Ctrl+C to stop")
    conversation_transcriber.start_transcribing_async()
    
    # Wait for completion or user interrupt
    try:
        while not done:
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("Stopping transcription...")
    finally:
        conversation_transcriber.stop_transcribing_async()


def main():
    """Main function to run the example"""
    print("Azure Speech-to-Text with Speaker Diarization")
    print("1. Transcribe from microphone")
    print("2. Transcribe from audio file")
    
    choice = input("Select an option (1 or 2): ")
    
    if choice == "1":
        # Get language code (optional)
        language = input("Enter language code (press Enter for Urdu/ur-IN): ") or "ur-IN"
        speech_to_text_with_diarization_from_mic(language)
    elif choice == "2":
        # Get file path
        file_path = input("Enter audio file path: ")
        if not os.path.exists(file_path):
            print(f"Error: File '{file_path}' does not exist")
            return
            
        # Get language code (optional)
        language = input("Enter language code (press Enter for Urdu/ur-IN): ") or "ur-IN"
        speech_to_text_with_diarization_from_file(file_path, language)
    else:
        print("Invalid option selected")


if __name__ == "__main__":
    main()

Azure Speech-to-Text with Speaker Diarization
1. Transcribe from microphone
2. Transcribe from audio file
Error: File '' does not exist


: 

## Google Cloud Speech-to-Text API

In [19]:
import io
import os
import subprocess
import tempfile
import shutil
import time
import logging
import glob # For finding chunk files
from google.cloud import speech_v1p1beta1 as speech
from google.api_core import exceptions as google_exceptions

In [20]:
DEFAULT_LANGUAGE_CODE = "ur-PK"  # Language for transcription
TARGET_SAMPLE_RATE_HZ = 16000    # Sample rate for chunks (should match your input FLAC)
CHUNK_DURATION_SECONDS = 55      # Duration of each audio chunk (max 59 for safety)
MAX_API_RETRIES = 3              # Max retries for a single chunk transcription
RETRY_INITIAL_DELAY_SECONDS = 5  # Initial delay between retries (will double each time)
OUTPUT_TRANSCRIPT_FILENAME = "full_transcription_from_local_chunks.txt"

In [21]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(module)s - %(funcName)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

In [22]:
class TranscriptionError(Exception):
    """Custom exception for transcription failures."""
    pass

def check_ffmpeg_installed():
    """Checks if ffmpeg is installed and accessible."""
    logger.info("Checking for ffmpeg installation...")
    try:
        subprocess.run(["ffmpeg", "-version"], capture_output=True, check=True, text=True)
        logger.info("ffmpeg found.")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        logger.error(f"ffmpeg not found or not executable. Please ensure ffmpeg is installed and in your system's PATH. Error: {e}")
        return False

def segment_audio_ffmpeg(input_audio_path, output_dir, chunk_duration_s, target_sample_rate_hz):
    """
    Segments the input audio file into smaller chunks using ffmpeg.
    Assumes input_audio_path is already in FLAC format at target_sample_rate_hz.

    Args:
        input_audio_path (str): Path to the long input audio file (FLAC).
        output_dir (str): Directory to save the audio chunks.
        chunk_duration_s (int): Desired duration of each chunk in seconds.
        target_sample_rate_hz (int): The sample rate of the input audio.

    Returns:
        list: A sorted list of paths to the created audio chunk files.
              Returns an empty list if segmentation fails.
    """
    logger.info(f"Starting audio segmentation for: {input_audio_path}")
    logger.info(f"Chunk duration: {chunk_duration_s}s, Output directory: {output_dir}")

    if not os.path.exists(input_audio_path):
        logger.error(f"Input audio file not found: {input_audio_path}")
        return []

    # Output chunk filename pattern
    chunk_filename_pattern = os.path.join(output_dir, f"chunk_%04d.flac")

    # ffmpeg command to segment the audio.
    # -i: input file
    # -f segment: output format is segment
    # -segment_time: duration of each segment
    # -c copy: copy the codec (assumes input is already FLAC and properly encoded)
    #          This is faster and avoids re-encoding if input is already good.
    # -ar: audio sample rate (to ensure consistency, though -c copy should preserve it)
    # -ac 1: audio channels (mono)
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_audio_path,
        "-f", "segment",
        "-segment_time", str(chunk_duration_s),
        "-c", "copy", # Assuming input is already FLAC, mono, 16000Hz
        chunk_filename_pattern,
    ]

    try:
        logger.info(f"Executing ffmpeg command: {' '.join(ffmpeg_command)}")
        process = subprocess.run(ffmpeg_command, capture_output=True, text=True, check=True)
        logger.info("ffmpeg segmentation successful.")
        logger.debug(f"ffmpeg stdout:\n{process.stdout}")
        logger.debug(f"ffmpeg stderr:\n{process.stderr}")
    except subprocess.CalledProcessError as e:
        logger.error(f"ffmpeg segmentation failed. Return code: {e.returncode}")
        logger.error(f"ffmpeg stdout:\n{e.stdout}")
        logger.error(f"ffmpeg stderr:\n{e.stderr}")
        return []
    except FileNotFoundError:
        logger.error("ffmpeg command not found. Ensure ffmpeg is installed and in PATH.")
        return []

    # Find all created chunk files and sort them
    # Using glob.glob to find files matching the pattern
    chunk_files = sorted(glob.glob(os.path.join(output_dir, "chunk_*.flac")))

    if not chunk_files:
        logger.warning("No chunk files were created by ffmpeg. Check ffmpeg output and input file.")
    else:
        logger.info(f"Created {len(chunk_files)} audio chunks.")

    return chunk_files

def transcribe_single_chunk_with_retry(
    speech_client,
    chunk_path,
    language_code,
    sample_rate_hz,
    max_retries=MAX_API_RETRIES,
    initial_delay_s=RETRY_INITIAL_DELAY_SECONDS
):
    """
    Transcribes a single audio chunk using Google Speech-to-Text with retries for transient errors.

    Args:
        speech_client: Initialized SpeechClient.
        chunk_path (str): Path to the audio chunk file.
        language_code (str): Language code for transcription.
        sample_rate_hz (int): Sample rate of the audio chunk.
        max_retries (int): Maximum number of retry attempts.
        initial_delay_s (int): Initial delay in seconds for retries (exponential backoff).

    Returns:
        str: The transcript of the chunk, or an empty string if transcription fails after retries.
    """
    logger.info(f"Transcribing chunk: {os.path.basename(chunk_path)}")

    if not os.path.exists(chunk_path):
        logger.error(f"Chunk file not found: {chunk_path}")
        return ""
    
    try:
        with io.open(chunk_path, "rb") as audio_file:
            content = audio_file.read()
    except IOError as e:
        logger.error(f"Could not read chunk file {chunk_path}: {e}")
        return ""

    audio_input = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.FLAC, # Assuming chunks are FLAC
        sample_rate_hertz=sample_rate_hz,
        language_code=language_code,
        enable_automatic_punctuation=True,
        # model="latest_short", # You can specify a model if needed
    )

    current_retry = 0
    delay = initial_delay_s
    while current_retry <= max_retries:
        try:
            response = speech_client.recognize(config=config, audio=audio_input)
            
            chunk_transcript = ""
            if response.results:
                for result in response.results:
                    chunk_transcript += result.alternatives[0].transcript + " "
                logger.info(f"Successfully transcribed chunk: {os.path.basename(chunk_path)}")
                return chunk_transcript.strip()
            else:
                logger.warning(f"No speech detected or transcription result for chunk: {os.path.basename(chunk_path)}")
                return "" # No result, no need to retry for this specific case

        except google_exceptions.RetryError as e: # Typically covers DeadlineExceeded, ServiceUnavailable
            logger.warning(f"API RetryError for chunk {os.path.basename(chunk_path)} (Attempt {current_retry+1}/{max_retries+1}): {e}. Retrying in {delay}s...")
        except google_exceptions.ServiceUnavailable as e:
            logger.warning(f"API ServiceUnavailable for chunk {os.path.basename(chunk_path)} (Attempt {current_retry+1}/{max_retries+1}): {e}. Retrying in {delay}s...")
        except google_exceptions.DeadlineExceeded as e:
            logger.warning(f"API DeadlineExceeded for chunk {os.path.basename(chunk_path)} (Attempt {current_retry+1}/{max_retries+1}): {e}. Retrying in {delay}s...")
        except google_exceptions.GoogleAPIError as e:
            logger.error(f"Non-retryable Google API error for chunk {os.path.basename(chunk_path)}: {e}")
            return "" # For other API errors (like PermissionDenied, InvalidArgument), don't retry

        time.sleep(delay)
        current_retry += 1
        delay *= 2  # Exponential backoff

    logger.error(f"Failed to transcribe chunk {os.path.basename(chunk_path)} after {max_retries} retries.")
    return ""


def transcribe_long_local_audio_by_chunking(
    local_long_audio_path,
    output_transcript_file=OUTPUT_TRANSCRIPT_FILENAME,
    language_code=DEFAULT_LANGUAGE_CODE,
    target_sample_rate=TARGET_SAMPLE_RATE_HZ,
    chunk_duration=CHUNK_DURATION_SECONDS
):
    """
    Transcribes a long local audio file by chunking it and processing each chunk.

    Args:
        local_long_audio_path (str): Path to the long input audio file (FLAC format recommended).
        output_transcript_file (str): Path to save the final combined transcript.
        language_code (str): BCP-47 language code for transcription.
        target_sample_rate (int): The sample rate your audio file is (and chunks will be).
        chunk_duration (int): Duration of each audio chunk in seconds.

    Returns:
        bool: True if transcription was successful (even if partially), False otherwise.
    """
    logger.info(f"--- Starting long local audio transcription process for: {local_long_audio_path} ---")

    if not check_ffmpeg_installed():
        return False # Prerequisite not met

    if not os.path.exists(local_long_audio_path):
        logger.error(f"Input audio file does not exist: {local_long_audio_path}")
        return False

    temp_chunk_dir = None
    try:
        # Create a temporary directory for audio chunks
        temp_chunk_dir = tempfile.mkdtemp(prefix="audio_chunks_")
        logger.info(f"Created temporary directory for chunks: {temp_chunk_dir}")

        # Step 1: Segment the audio file into chunks
        chunk_file_paths = segment_audio_ffmpeg(
            local_long_audio_path, temp_chunk_dir, chunk_duration, target_sample_rate
        )

        if not chunk_file_paths:
            logger.error("Audio segmentation failed or produced no chunks. Aborting.")
            raise TranscriptionError("Audio segmentation failed.")

        # Step 2: Initialize Speech-to-Text client
        try:
            speech_client = speech.SpeechClient()
            logger.info("Google Cloud Speech client initialized successfully.")
        except google_exceptions.DefaultCredentialsError as e:
            logger.error(f"Google Cloud Default Credentials not found. Ensure you've run 'gcloud auth application-default login'. Error: {e}")
            raise TranscriptionError("Authentication failed.") from e
        except Exception as e: # Catch any other client initialization errors
            logger.error(f"Failed to initialize Google Cloud Speech client: {e}")
            raise TranscriptionError("Speech client initialization failed.") from e

        # Step 3: Transcribe each chunk
        all_transcripts_segments = []
        processed_chunks = 0
        for i, chunk_path in enumerate(chunk_file_paths):
            logger.info(f"Processing chunk {i + 1}/{len(chunk_file_paths)}: {os.path.basename(chunk_path)}")
            transcript_segment = transcribe_single_chunk_with_retry(
                speech_client, chunk_path, language_code, target_sample_rate
            )
            if transcript_segment: # Append even if it's just spaces, to maintain order if needed
                all_transcripts_segments.append(transcript_segment)
            processed_chunks +=1
            # Optional: Progress update
            if processed_chunks % 5 == 0 or processed_chunks == len(chunk_file_paths):
                 logger.info(f"Progress: {processed_chunks}/{len(chunk_file_paths)} chunks submitted for transcription.")


        # Step 4: Combine transcripts and save to file
        if not all_transcripts_segments:
            logger.warning("No transcription segments were generated from any chunk.")
            # Depending on requirements, this could be an error or just an empty result.
            # For now, we'll write an empty file.
            final_transcript = ""
        else:
            final_transcript = "\n".join(all_transcripts_segments).strip() # Use newline, or " " for continuous text

        logger.info(f"Writing combined transcript to: {output_transcript_file}")
        try:
            with open(output_transcript_file, "w", encoding="utf-8") as f:
                f.write(final_transcript)
            logger.info("Successfully wrote transcript to file.")
        except IOError as e:
            logger.error(f"Failed to write transcript to file {output_transcript_file}: {e}")
            # Don't raise TranscriptionError here, as transcription might have partially succeeded.
            # The transcript is in final_transcript variable.

        if not final_transcript and len(chunk_file_paths) > 0 :
            logger.warning("Transcription process completed, but the resulting transcript is empty.")
        elif final_transcript:
             logger.info("--- Transcription process completed successfully. ---")

        return True

    except TranscriptionError as e: # Catch custom errors for aborting conditions
        logger.error(f"A critical error occurred during transcription: {e}")
        return False
    except Exception as e: # Catch-all for unexpected errors
        logger.exception(f"An unexpected error occurred: {e}") # .exception includes stack trace
        return False
    finally:
        # Step 5: Cleanup temporary directory
        if temp_chunk_dir and os.path.exists(temp_chunk_dir):
            try:
                shutil.rmtree(temp_chunk_dir)
                logger.info(f"Successfully cleaned up temporary chunk directory: {temp_chunk_dir}")
            except OSError as e:
                logger.error(f"Error cleaning up temporary directory {temp_chunk_dir}: {e}")

In [23]:
logger.info("Script started.")

2025-05-12 10:55:24 - INFO - 1896180177 - <module> - Script started.


In [ ]:
input_audio_file_path = r"D:\Taleemabad\Digital Coach\Dataset\Urban II\918-IMCG I-8-4\Obs 1-918_converted.flac"

In [25]:
success = transcribe_long_local_audio_by_chunking(
            local_long_audio_path=input_audio_file_path,
            output_transcript_file=OUTPUT_TRANSCRIPT_FILENAME,
            language_code=DEFAULT_LANGUAGE_CODE,
            target_sample_rate=TARGET_SAMPLE_RATE_HZ,
            chunk_duration=CHUNK_DURATION_SECONDS
        )

if success:
    logger.info(f"Script finished. Check '{OUTPUT_TRANSCRIPT_FILENAME}' for results.")
else:
    logger.error("Script finished with errors. Please review the logs.")

2025-05-12 10:57:06 - INFO - 2400712708 - transcribe_long_local_audio_by_chunking - --- Starting long local audio transcription process for: D:\Taleemabad\Digital Coach\Dataset\Urban II\918-IMCG I-8-4\Obs 1-918_converted.flac ---
2025-05-12 10:57:06 - INFO - 2400712708 - check_ffmpeg_installed - Checking for ffmpeg installation...
2025-05-12 10:57:06 - INFO - 2400712708 - check_ffmpeg_installed - ffmpeg found.
2025-05-12 10:57:06 - INFO - 2400712708 - transcribe_long_local_audio_by_chunking - Created temporary directory for chunks: C:\Users\mypc\AppData\Local\Temp\audio_chunks_04yk84vv
2025-05-12 10:57:06 - INFO - 2400712708 - segment_audio_ffmpeg - Starting audio segmentation for: D:\Taleemabad\Digital Coach\Dataset\Urban II\918-IMCG I-8-4\Obs 1-918_converted.flac
2025-05-12 10:57:06 - INFO - 2400712708 - segment_audio_ffmpeg - Chunk duration: 55s, Output directory: C:\Users\mypc\AppData\Local\Temp\audio_chunks_04yk84vv
2025-05-12 10:57:06 - INFO - 2400712708 - segment_audio_ffmpeg - 

# OPENAI

In [16]:
!pip install pyaudioop

ERROR: Could not find a version that satisfies the requirement pyaudioop (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for pyaudioop


In [30]:
!pip install pydub openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Simple Transcription into Urdu

In [ ]:
import os
import tempfile
from pydub import AudioSegment
from openai import OpenAI
from dotenv import load_dotenv

# Load API Key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def simple_chunk_audio(audio_path, chunk_size_ms=30000, overlap_ms=5000):
    """Split audio into chunks with minimal dependencies"""
    print(f"Loading audio file: {audio_path}")
    
    try:
        # Load audio file
        audio = AudioSegment.from_file(audio_path)
        
        # Get audio duration
        duration_ms = len(audio)
        
        # Calculate step size
        step_ms = chunk_size_ms - overlap_ms
        
        # Create chunks
        chunks = []
        for start_ms in range(0, duration_ms, step_ms):
            end_ms = min(start_ms + chunk_size_ms, duration_ms)
            chunk = audio[start_ms:end_ms]
            chunks.append(chunk)
            
        print(f"Created {len(chunks)} chunks from {duration_ms/1000:.1f}s audio")
        return chunks
    except Exception as e:
        print(f"Error loading audio: {e}")
        print("Make sure ffmpeg is installed for handling audio files")
        raise

def transcribe_chunks(chunks, client, language="ur"):
    """Transcribe each chunk using the OpenAI API"""
    transcriptions = []
    
    for i, chunk in enumerate(chunks):
        print(f"Transcribing chunk {i+1}/{len(chunks)}...")
        
        # Create temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        temp_path = temp_file.name
        temp_file.close()
        
        try:
            # Export chunk to temporary file
            chunk.export(temp_path, format="mp3")
            
            # Transcribe using OpenAI API
            with open(temp_path, 'rb') as audio_file:
                response = client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    language=language
                )
                
            transcriptions.append(response.text)
            print(f"Chunk {i+1} done: {response.text[:30]}...")
            
        except Exception as e:
            print(f"Error processing chunk {i+1}: {e}")
            transcriptions.append("[Error transcribing this segment]")
            
        finally:
            # Clean up
            if os.path.exists(temp_path):
                os.remove(temp_path)
    
    return transcriptions

# Main execution
def main():
    # Path to your audio file
    audio_path = r"Dataset\Urban II\918-IMCG I-8-4\Obs 1-918.m4a"
    
    # Use a try-except block to catch any errors
    try:
        print("Splitting audio...")
        chunks = simple_chunk_audio(audio_path)
        
        print("Transcribing chunks...")
        transcriptions = transcribe_chunks(chunks, client)
        
        # Combine transcriptions
        full_text = " ".join(transcriptions)
        
        # Save the result
        with open("transcription.txt", "w", encoding="utf-8") as f:
            f.write(full_text)
        
        print("Transcription complete and saved to transcription.txt")
        print(full_text[:500] + "...")  # Show the first 500 characters
    
    except Exception as e:
        print(f"An error occurred: {e}")
        # Provide a backup solution
        try_alternative_solution()

def try_alternative_solution():
    """Backup solution that tries to transcribe the whole file at once"""
    print("\nTrying alternative approach: direct transcription...")
    audio_path = r"Dataset\Urban II\918-IMCG I-8-4\Obs 1-918.m4a"
    
    try:
        with open(audio_path, "rb") as audio_file:
            response = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="ur"
            )
        
        # Save transcription
        with open("transcription_direct.txt", "w", encoding="utf-8") as f:
            f.write(response.text)
        
        print("Direct transcription successful! Saved to transcription_direct.txt")
        print(response.text[:500] + "...")
    
    except Exception as e:
        print(f"Direct transcription also failed: {e}")
        print("\nIf both methods fail, you may need to:")
        print("1. Ensure ffmpeg is installed (needed for pydub)")
        print("2. Manually split your audio file using software like Audacity")
        print("3. Transcribe each part separately")

if __name__ == "__main__":
    main()

Splitting audio...
Loading audio file: Dataset\Urban II\918-IMCG I-8-4\Obs 1-918.m4a
Created 67 chunks from 1651.2s audio
Transcribing chunks...
Transcribing chunk 1/67...
Chunk 1 done: آپنے بکس بنائیں اوپر یہ پیری پ...
Transcribing chunk 2/67...
Chunk 2 done: یا بیٹھیں گے ابیں تھوڑا سا یہ ...
Transcribing chunk 3/67...
Chunk 3 done: اب جب ہم نے کلاغ بنائے اور ہم ...
Transcribing chunk 4/67...
Chunk 4 done: GG اور ہم نے ہنٹس بھی بنانی سک...
Transcribing chunk 5/67...
Chunk 5 done: اور اس کے اندروں سے تک تری ہین...
Transcribing chunk 6/67...
Chunk 6 done: جو minute hand ہے big hand وہ ...
Transcribing chunk 7/67...
Chunk 7 done: دو دو منٹ ہنڈ دو ہار ہند ہے ای...
Transcribing chunk 8/67...
Chunk 8 done: اگر آپ مجھے بتائیں گے اگر اس ک...
Transcribing chunk 9/67...
Chunk 9 done: اگلے بیٹے کو ڈوائٹ کریں گے اور...
Transcribing chunk 10/67...
Chunk 10 done: ...
Transcribing chunk 11/67...
Chunk 11 done: اب آپ دیکھیں ایک دن دو حصوں می...
Transcribing chunk 12/67...
Chunk 12 done: بابا جی کا پھ

## What if we tranlate it into English?

In [1]:
import os
import tempfile
from pydub import AudioSegment
from openai import OpenAI
from dotenv import load_dotenv

# Load API Key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def chunk_audio(audio_path, chunk_size_ms=30000, overlap_ms=5000):
    """Split audio into chunks"""
    print(f"Loading audio file: {audio_path}")
    audio = AudioSegment.from_file(audio_path)
    duration_ms = len(audio)
    step_ms = chunk_size_ms - overlap_ms
    
    chunks = []
    for start_ms in range(0, duration_ms, step_ms):
        end_ms = min(start_ms + chunk_size_ms, duration_ms)
        chunk = audio[start_ms:end_ms]
        chunks.append(chunk)
    
    print(f"Created {len(chunks)} chunks from {duration_ms/1000:.1f}s audio")
    return chunks

def translate_chunks(chunks, client):
    """Translate each chunk from Urdu to English"""
    translations = []
    
    for i, chunk in enumerate(chunks):
        print(f"Translating chunk {i+1}/{len(chunks)}...")
        
        # Create temporary file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        temp_path = temp_file.name
        temp_file.close()
        
        try:
            # Export chunk to temporary file
            chunk.export(temp_path, format="mp3")
            
            # Translate using OpenAI API
            with open(temp_path, 'rb') as audio_file:
                response = client.audio.translations.create(
                    model="whisper-1",
                    file=audio_file
                )
                
            translations.append(response.text)
            print(f"Chunk {i+1} done: {response.text[:30]}...")
            
        except Exception as e:
            print(f"Error processing chunk {i+1}: {e}")
            translations.append("[Error translating this segment]")
            
        finally:
            # Clean up
            if os.path.exists(temp_path):
                os.remove(temp_path)
    
    return translations

# Main execution
audio_path = r"Dataset\Urban II\918-IMCG I-8-4\Obs 1-918.m4a"


print("Splitting audio...")
chunks = chunk_audio(audio_path)

print("Translating chunks...")
translations = translate_chunks(chunks, client)

# Combine translations
full_text = " ".join(translations)

# Save the result
with open("english_translation.txt", "w", encoding="utf-8") as f:
    f.write(full_text)

print("Translation complete and saved to english_translation.txt")
print(full_text[:500] + "...")  # Show the first 500 characters

Splitting audio...
Loading audio file: Dataset\Urban II\918-IMCG I-8-4\Obs 1-918.m4a
Created 67 chunks from 1651.2s audio
Translating chunks...
Translating chunk 1/67...
Chunk 1 done: We have made a lot of blogs an...
Translating chunk 2/67...
Chunk 2 done: Now, let's take a look at what...
Translating chunk 3/67...
Chunk 3 done: When we made the clock and saw...
Translating chunk 4/67...
Chunk 4 done: And we also learned how to mak...
Translating chunk 5/67...
Chunk 5 done: And there are three hands in i...
Translating chunk 6/67...
Chunk 6 done: Where is the minute hand? Here...
Translating chunk 7/67...
Chunk 7 done: Eleven. Twelve. Our end is at ...
Translating chunk 8/67...
Chunk 8 done: If you count your hours, there...
Translating chunk 9/67...
Chunk 9 done: We will divide it and you will...
Translating chunk 10/67...
Chunk 10 done: Eleven, twelve. Twelve. Good a...
Translating chunk 11/67...
Chunk 11 done: Now we will see that one day i...
Translating chunk 12/67...
Chunk 12 do

# Whisper Medium Urdu
### Done on the EC2 Machine

# Assembly AI NANO MODEL

In [13]:
import assemblyai as aai

In [14]:
load_dotenv()
api_key = os.getenv("ASSEMBLY_KEY")
aai.settings.api_key = api_key

In [21]:
audio_file = r"Dataset\Urban II\918-IMCG I-8-4\Debrief 1-918.m4a"

In [27]:
config = aai.TranscriptionConfig(speech_model=aai.SpeechModel.nano, language_code="ur")

In [28]:
transcript = aai.Transcriber(config=config).transcribe(audio_file)

In [29]:
if transcript.status == "error":
  raise RuntimeError(f"Transcription failed: {transcript.error}")

In [30]:
print(f" \nFull Transcript: \n\n{transcript.text}")

 
Full Transcript: 

مجھے کلاٹے złاتےLeftے ہیں؟ میرے بیخی بڑے انڈوائی کرتی ہے بیخی انڈوائی کلاٹی کرتی سکتا ہوں اے لئے اہم انiinوز کیلیے سبھی ہے بھی انہوں کے لیے ایک نئی چیزٹی کیا ہے وہ اسی ہی کنک کر پا رہے تھے کنک کر پا رہے تھے تو یہ ایک دو بھی جو مصنفت دیتا ہے کہ وہ پھر سمجھا تھا کہ اگر ہم شام کی اس کی بات کریں تو وہ پیم ہو گا اور اگر ہم جو بہت اس کی بات کریں تو وہ ایم ہو گا لیکن وہ بہت اچھے سب کنسرٹ کو کر لیا ویسے کہ ان پر سب کو پتھلا گیا تھا کہ ان کی سب کنگ دوکل کیا اکثر لیکھ ہوتا ہے کہ اس سے اس کے بچے نہیں سمجھتے کیونکہ اس سے اس سے اکہ مجیس سے انہوں نے سمجھا تھا کہ تو ان کے شخص میں کھو ایسے لگاتا ہے ان کا سب کے صورت ہے تو اپنے ذنان کے شخص میں ایم ہو گئے مرحبا ہوں کہ اُن نے نتی کانک کر دوکل کیا نا ، میں مجھے کیاFlash کرتا ہے ایک نیسرٹ میں ایک نیسرٹ میں اُتنا پوچھا اگر جب کلین سی کلینگ پیچھا ہے وہ دور کیا ہے نائی الحیت جاتا ہوا ہے وہ اگر پیچھنا پہلے جاتا ہے اور ایروپلینہ نہیں ہم چاہتے ہیں وہ انہوں نے ایک زیادہ ملشہ کر رہی ہے جو بھی ملشہ کر رہا ہی جاک کیوں پہلی تخزون ہم کرار اگر ہی دی

In [ ]:
with open('assemblyai_transcript.txt', 'w', encoding='utf-8') as f:
    f.write(transcript.text + "\n")